# **Importing libraries**

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import sys
import librosa
import torchvision
import numpy as np

import matplotlib.pyplot as plt
import IPython.display as ipd
import pandas as pd

from tqdm import tqdm
%matplotlib inline

from torchsummary import summary


# Get Computtional Device

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Importing the Dataset

In [3]:
# Percorso base del dataset RAVDESS su Kaggle
# (Kaggle salva automaticamente i dataset nella cartella /kaggle/input/)
DATASET_PATH = "/kaggle/input/datasets/uwrfkaggler/ravdess-emotional-speech-audio"

# Mappatura delle emozioni (il 3° numero nel nome del file RAVDESS)
emotion_map = {
    '01': 'Neutral',
    '02': 'Calm',
    '03': 'Happy',
    '04': 'Sad',
    '05': 'Angry',
    '06': 'Fearful',
    '07': 'Disgust',
    '08': 'Surprised'
}

print("Librerie importate con successo!")

Librerie importate con successo!


In [4]:
import os
import torch
import torchaudio
from torch.utils.data import Dataset

class RavdessDataset(Dataset):
    def __init__(self, root_dir=DATASET_PATH, subset="training", emotions=None):
        """
        emotions: lista di stringhe con i codici delle emozioni da tenere (es. ["03", "04"] per Felice e Triste).
                  Se None, prende tutte le emozioni.
        """
        self.root_dir = root_dir
        self.emotions = emotions if emotions is not None else []
        self._walker = [] # Conterrà i percorsi dei file audio selezionati

        # 1. Trova tutti i file .wav nelle sottocartelle
        all_files = []
        for dirpath, _, filenames in os.walk(root_dir):
            for f in filenames:
                if f.endswith(".wav"):
                    all_files.append(os.path.join(dirpath, f))

        # 2. Filtra e suddividi il dataset
        for file_path in all_files:
            filename = os.path.basename(file_path)
            
            # Rimuove ".wav" e divide la stringa usando i trattini
            parts = filename.replace(".wav", "").split("-")
            
            # Controlla che il file sia effettivamente nel formato RAVDESS (7 parti)
            if len(parts) != 7:
                continue
                
            emotion = parts[2]
            actor = int(parts[6]) # Convertiamo in intero per fare i confronti (es. "05" diventa 5)

            # Filtra per etichetta (emozione)
            if len(self.emotions) > 0 and emotion not in self.emotions:
                continue # Salta questo file e passa al prossimo

            # Suddivide per Actor (Previene il data leakage)
            if subset == "training" and actor <= 18:
                self._walker.append(file_path)
            elif subset == "validation" and 19 <= actor <= 22:
                self._walker.append(file_path)
            elif subset == "testing" and actor >= 23:
                self._walker.append(file_path)
            elif subset is None:
                self._walker.append(file_path) # Carica tutto se subset non è specificato

    def __len__(self):
        # PyTorch ha bisogno di sapere quanto è grande il dataset
        return len(self._walker)

    def __getitem__(self, idx):
        # Cosa succede quando chiedi un elemento al dataset (es. dataset[0])
        file_path = self._walker[idx]
        
        # Carica l'audio
        waveform, sample_rate = torchaudio.load(file_path)
        
        # Estrai l'emozione dal nome per usarla come etichetta (Label)
        filename = os.path.basename(file_path)
        emotion_label = filename.split("-")[2]
        
        # Converte l'etichetta in un tensore numerico (sottraendo 1 per partire da 0)
        # RAVDESS va da 01 a 08. Sottraendo 1 avremo classi da 0 a 7, perfette per PyTorch.
        label_tensor = torch.tensor(int(emotion_label) - 1, dtype=torch.long)
        
        return waveform, sample_rate, label_tensor

In [5]:
# Voglio solo Felicità (03) e Rabbia (05)
emozioni_scelte = ["03", "05"]

train_dataset = RavdessDataset(subset="training", emotions=emozioni_scelte)
val_dataset = RavdessDataset(subset="validation", emotions=emozioni_scelte)
test_dataset = RavdessDataset(subset="testing", emotions=emozioni_scelte)

print(f"File di training trovati: {len(train_dataset)}")
print(f"File di validation trovati: {len(val_dataset)}")

File di training trovati: 576
File di validation trovati: 128
